In [ ]:
from mysklearn.mypytable import MyPyTable
import mysklearn.myutils as myutils
from mysklearn.myclassifiers import MyRandomForestClassifier

table = MyPyTable().load_from_file("cleaned_tracks.csv")

def categorize_popularity(p):
    p = int(p)
    if p <= 33:
        return "Low"
    elif p <= 66:
        return "Medium"
    else:
        return "High"

pop_idx = table.column_names.index("popularity")
y = [categorize_popularity(row[pop_idx]) for row in table.data]

explicit_idx = table.column_names.index("explicit")
duration_idx = table.column_names.index("duration_ms")
dance_idx = table.column_names.index("danceability")
energy_idx = table.column_names.index("energy")
tempo_idx = table.column_names.index("tempo")
loudness_idx = table.column_names.index("loudness")

X = []
for row in table.data:
    X.append([
        row[explicit_idx],
        row[duration_idx],
        row[dance_idx],
        row[energy_idx],
        row[tempo_idx],
        row[loudness_idx]
    ])

rf = MyRandomForestClassifier()
rf.fit(X, y)

# Predict an example song
example_song = [
    1,         # explicit
    150000,    # duration_ms
    0.25,      # danceability
    0.50,      # energy
    120.0      # tempo
    -8.0       # loudness
]

print("Prediction:", rf.predict([example_song]))

In [ ]:
import mysklearn.myevaluation as myevaluation

rf_acc, rf_err, rf_tp, rf_p = myutils.cross_val_predict(X, y, MyRandomForestClassifier)

print(f"rf acc: {rf_acc:.2f} rf err: {rf_err:.2f}")

rf_prec = myevaluation.binary_precision_score(rf_tp, rf_p)

print(f"rf precision: {rf_prec:.2f}")

rf_recall = myevaluation.binary_recall_score(rf_tp, rf_p)

print(f"rf recall: {rf_recall:.2f}")

rf_f1 = myevaluation.binary_f1_score(rf_tp, rf_p)

print(f"rf f1: {rf_f1:.2f}")

In [ ]:

labels = ["Low", "Medium", "High"]
matrix = [[0 for _ in labels] for _ in labels]
label_to_idx = {label: i for i, label in enumerate(labels)}

for t, p in zip(rf_tp, rf_p):
    i = label_to_idx[t]
    j = label_to_idx[p]
    matrix[i][j] += 1

print_confusion_matrix(labels, matrix, "Random Forest Confusion Matrix")